In [ ]:
!pip install datasets
# !pip install transformers
!pip install transformers==3.4.0
#I did this because I was getting some error (don't remember, it's been a week since I ran these) and stackoverflow told me that the error was
#because transformers are not backwards compatible and doing this did fix that error
!pip install evaluate
#NOTE: This notebook is messy because I went pretty deep in the rabbit hole of debugging but I've tried to explain what was going on in my head at the time
#But, none of the stuff here actually works, hence the name of this notebook

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 5.1 MB/s 
     |████████████████████████████████| 182 kB 64.2 MB/s 
     |████████████████████████████████| 212 kB 64.4 MB/s 
     |████████████████████████████████| 115 kB 68.7 MB/s 
     |████████████████████████████████| 127 kB 64.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.1 MB/s 
     |████████████████████████████████| 1.3 MB 46.7 MB/s 
     |████████████████████████████████| 880 kB 59.3 MB/s 
     |████████████████████████████████| 2.9 MB 49.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=abefe0c01a8cbdec99dbb7b58d2f

In [ ]:
from datasets import load_dataset
#For Task 4 on the Children's Book Test dataset, the authors chose NE and CN only
datasetCBTNE = load_dataset("cbt", "NE")
datasetCBTCN = load_dataset("cbt", "CN")
# # you can use any of the following config names as a second argument:
# "CN", "NE", "P", "V", 
# "raw"



In [ ]:
# print("CBT", datasetCBTNE)
# print("CBT", datasetCBTCN)

In [ ]:
from datasets import Dataset, load_metric, ClassLabel
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import os
import json
import evaluate

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
	logits,labels = eval_pred
	predictions = np.argmax(logits, axis=-1)
	result = metric.compute(predictions=predictions, references=labels)
	return result

# CBT sentence level auto QA - 
datasetCBTNE_train = datasetCBTNE["train"].train_test_split(test_size=0.15)
datasetCBTNE_test = datasetCBTNE["test"]
# datasetCBTCN_train = datasetCBTCN["train"]
# datasetCBTCN_test = datasetCBTCN["test"]
print(datasetCBTNE_train, datasetCBTNE_test)

DatasetDict({
    train: Dataset({
        features: ['sentences', 'question', 'answer', 'options'],
        num_rows: 92411
    })
    test: Dataset({
        features: ['sentences', 'question', 'answer', 'options'],
        num_rows: 16308
    })
}) Dataset({
    features: ['sentences', 'question', 'answer', 'options'],
    num_rows: 2500
})


In [ ]:
print("building tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

#I tried to make it so that the dataset would have text and label columns so that DistilBERT doesn't get confused and knows what to do (classification)
#But it did not work

# datasetCBTNE_train = datasetCBTNE_train.rename_column('answer', 'labels')
# datasetCBTNE_test = datasetCBTNE_test.rename_column('answer', 'labels')
# print(datasetCBTNE_train, datasetCBTNE_test)
# datasetCBTNE_train = datasetCBTNE_train.rename_column('question', 'text')
# datasetCBTNE_test = datasetCBTNE_test.rename_column('question', 'text')
def tokenize_function(examples):
	token = tokenizer(examples["text"], padding="max_length", truncation=True)
	return token


building tokenizer...
DatasetDict({
    train: Dataset({
        features: ['sentences', 'question', 'labels', 'options'],
        num_rows: 92411
    })
    test: Dataset({
        features: ['sentences', 'question', 'labels', 'options'],
        num_rows: 16308
    })
}) Dataset({
    features: ['sentences', 'question', 'labels', 'options'],
    num_rows: 2500
})


In [ ]:

tokenized_datasetCBTNE_train = datasetCBTNE_train.map(tokenize_function, batched=True)
print(tokenized_datasetCBTNE_train)


  0%|          | 0/93 [00:00<?, ?ba/s]

  0%|          | 0/17 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['sentences', 'text', 'labels', 'options', 'input_ids', 'attention_mask'],
        num_rows: 92411
    })
    test: Dataset({
        features: ['sentences', 'text', 'labels', 'options', 'input_ids', 'attention_mask'],
        num_rows: 16308
    })
})


In [ ]:
#for converting string labels to int and vice-versa

label_names = datasetCBTNE_train['train'].unique('labels')
label_dict = {}
for i in range(len(label_names)):
  label_dict[i] = label_names[i]

print(label_dict)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length")
id2label = {k:l for k, l in enumerate(label_names)}
label2id = {l:k for k, l in enumerate(label_names)}

#Was getting an error that inputids and attentionmask is missing so added this to fix that but did not need it later on

# columns_to_return = ['input_ids', 'labels', 'attention_mask']
# tokenized_datasetCBTNE_train['train'].set_format(type='torch', columns=columns_to_return)
# tokenized_datasetCBTNE_train['test'].set_format(type='torch', columns=columns_to_return)
# print(tokenized_datasetCBTNE_train['train'], tokenized_datasetCBTNE_train['test'])

{0: 'Eisenkopf', 1: 'Katherine', 2: 'Alec', 3: 'Chipmunk', 4: 'Saxons', 5: 'Mollie', 6: 'Brown', 7: 'Unicorn', 8: 'Peter', 9: 'Joey', 10: 'John', 11: 'Pye', 12: 'Jem', 13: 'Anne', 14: 'Alice', 15: 'Jo', 16: 'Mouse', 17: 'Blythe', 18: 'World', 19: 'Lady', 20: 'morning', 21: 'evening', 22: 'Dick', 23: 'Thorny', 24: 'Rose-Leaf', 25: 'Cinderella', 26: 'Barr', 27: 'Danby', 28: 'Tom', 29: 'Meg', 30: 'Hans', 31: 'Nature', 32: 'Makoma', 33: 'Hannah', 34: 'Annie', 35: 'Possum', 36: 'Gilbert', 37: 'Charlotta', 38: 'Margaret', 39: 'Meredith', 40: 'peris', 41: 'godmother', 42: 'days', 43: 'Rabbit', 44: 'Queen', 45: 'Forest', 46: 'Bagheera', 47: 'fire', 48: 'Minot', 49: 'Hester', 50: 'Helen', 51: 'Lavinia', 52: 'Rusty', 53: 'Bennett', 54: 'Douglas', 55: 'Diana', 56: 'Costan', 57: 'Shirley', 58: 'Germany', 59: 'Alan', 60: 'Melissa', 61: 'Ted', 62: 'Ruth', 63: 'Boxer', 64: 'Jims', 65: 'swineherd', 66: 'Ingleside', 67: 'Paul', 68: 'Sally', 69: 'Fox', 70: 'Campbell', 71: 'Aunty', 72: 'grown-ups', 73: '

In [ ]:

# modelCBTNE = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = len(label_names))
modelCBTNE = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", id2label=id2label, label2id=label2id)
# modelCBTNE.cuda()
training_args = TrainingArguments(
	output_dir="/content/drive/My Drive/ColabNotebooks/model out/",
	# optim = "adamw_torch", 
	evaluation_strategy="steps", 
	#num_train_epochs = 10,
	metric_for_best_model = "accuracy",
	greater_is_better = True,
  # warmup_steps = 5000,
	# save_steps=5000,
	# max_steps = 25000,
	# eval_steps = 5000,
  warmup_steps = 50,
	save_steps=50,
	max_steps = 100,
	eval_steps = 50,
	per_device_train_batch_size=8, 
	per_device_eval_batch_size=8,
	weight_decay=0.01,
	# learning_rate = 2e-5,#, #defaults to 5e-5
	learning_rate = 5e-6,
	load_best_model_at_end = True,
	# lr_scheduler_type = "cosine"
	# report_to="wandb"
	)

trainer1 = Trainer(
		model=modelCBTNE,
		args=training_args,
		train_dataset=tokenized_datasetCBTNE_train["train"],
    eval_dataset=tokenized_datasetCBTNE_train["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
		compute_metrics=compute_metrics
)
trainer1.train()



Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

ValueError: ignored

In [ ]:
tokenized_datasetCBTNE_test = datasetCBTNE_test.map(tokenize_function, batched=True)

prediction1 = trainer1.predict(tokenized_datasetCBTNE_test)
print(prediction1)

In [ ]:
tokenizer1 = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=False)

#Made another tokenizer function in hopes that it might change something, it did not
def tokenize_function_qa(examples):
    text = []
    text.append(datasetCBTNE_train["sentences"])
    text.append('[SEP]')
    text.append(datasetCBTNE_train["question"])
    # questions = datasetCBTNE_train["question"]
    # # questions = ','.join(map(str, questions))
    # sentences = [q for q in datasetCBTNE_train["sentences"] if q is not None]
    # # options = [q for q in datasetCBTNE_train["options"] if q is not None]
    # # sentences = ','.join(map(str, sentences))
    inputs = tokenizer1(text,
        truncation=True,
        # stride=128,
        # return_overflowing_tokens=True,
        # return_offsets_mapping=True,
        padding="max_length"
    )
    return inputs



Dataset({
    features: ['sentences', 'question', 'answer', 'options'],
    num_rows: 108719
})


In [ ]:
from transformers import DistilBertForQuestionAnswering
modelCBT = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = 10)
tokenized_datasetCBTNE_train = datasetCBTNE_train.map(tokenize_function_qa, batched=True)
tokenized_datasetCBTCN_train = datasetCBTCN_train.map(tokenize_function_qa, batched=True)
print(tokenized_datasetCBTCN_train)
print(tokenized_datasetCBTNE_train)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

  0%|          | 0/109 [00:00<?, ?ba/s]

ValueError: ignored

In [ ]:
#Tried to keep data in memory and read it from memory instead of loading it from huggingface as I saw this method online
#Absolutely terrible method, the dataset is very big and Colab's RAM, even with Pro, is not anywhere near enough

# !wget -N http://www.thespermwhale.com/jaseweston/babi/CBTest.tgz
# !tar xf CBTest.tgz

In [ ]:
# # %cd CBTest
# # %cd data
# !ls

cache_dir		    cbtest_P_train.txt
CBTest			    cbtest_P_valid_2000ex.txt
cbtest_CN_test_2500ex.txt   CBTest.tgz
cbtest_CN_train.txt	    cbtest_V_test_2500ex.txt
cbtest_CN_valid_2000ex.txt  cbtest_V_train.txt
cbtest_NE_test_2500ex.txt   cbtest_V_valid_2000ex.txt
cbtest_NE_train.txt	    cbt_test.txt
cbtest_NE_valid_2000ex.txt  cbt_train.txt
cbtest_P_test_2500ex.txt    cbt_valid.txt


So I believed that if I changed my input to the transformer model so that it contains only the labels and text (concatinating everything except the correct answer), it should work. I tried to do so but I was stuck in the cycle of fixing an error and then getting another error so on and so forth. 

In [ ]:
import re
import pickle
#I gave up when using pickle for serializing and deserializing did not solve my issues and gave me more errors

#Function to pre-process the data which was mounted from my google drive/read from memory

def read_cbt_data(path, limit=None):
    with open(path, 'r') as f:
        stories = []
        context = []
        temp = []
        lab = []
        for line in f:
            line = line.replace('\n', '')
            if line == '':
                continue
            m = re.match(r'[0-9]* ', line).end()
            line_no = int(line[:m-1])
            sentence = line[m:]
            if line_no == 21: # process query.
                sentence = sentence.split('\t')
                query = sentence[0].strip().split(' ')
                answer = sentence[1].strip()
                candidate = sentence[3].strip().split('|')
                candidate = [c for c in candidate if c]
                while len(candidate) < 10:
                    candidate.append('<null>')
                assert(len(candidate) == 10)
                if context is not None:
                  temp.append(context)
                  temp.append('[SEP]')
                if query is not None:
                  temp.append(query)
                  temp.append('[SEP]')
                if answer is not None:
                  lab.append(answer)
                # story = dict()
                # story['context'] = list(str(context))
                # story['query'] = list(str(query))
                # story['candidate'] = list(str(candidate))
                # story['text'] = list(temp)
                # story['labels'] = list(answer)
                # story = {
                #     'context': context,
                #     'query': query,
                #     'labels': answer,
                #     'candidate': candidate
                #     # 'labels': lab,
                #     # 'text': text
                # }
                assert(len(context) == 20)
                # stories.append(story)
                if limit and len(stories) > limit:
                    break
                context = []
                temp = []
                lab = []
            else:
                context.append(sentence.strip().split(' '))
        stories
        stories = pd.DataFrame.from_dict(stories)
        stories.to_pickle("/tmp/stories.pickle")
        with open("/tmp/stories.pickle", "rb") as fin:
            stories1 = pickle.load(fin)
        return Dataset.from_pandas(stories1)
        # return stories1
        # return stories

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

CBTCN_train_path = "/content/drive/My Drive/data/cbtest_CN_train.txt"
CBTCN_validate_path = "/content/drive/My Drive/data/cbtest_CN_valid_2000ex.txt"
CBTCN_test_path = "/content/drive/My Drive/data/cbtest_CN_test_2500ex.txt"

CBTNE_train_path = "/content/drive/My Drive/data/cbtest_NE_train.txt"
CBTNE_validate_path = "/content/drive/My Drive/data/cbtest_NE_valid_2000ex.txt"
CBTNE_test_path = "/content/drive/My Drive/data/cbtest_NE_test_2500ex.txt"

Mounted at /content/drive


In [ ]:
datasetCBT_CN_train = read_cbt_data(CBTCN_train_path)
datasetCBT_CN_validate = read_cbt_data(CBTCN_validate_path)
datasetCBT_CN_test = read_cbt_data(CBTCN_test_path)

# datasetCBT_NE_train = read_cbt_data("cbtest_NE_train.txt")
# datasetCBT_NE_validate = read_cbt_data("cbtest_NE_valid_2000ex.txt")
# datasetCBT_NE_test = read_cbt_data("cbtest_NE_test_2500ex.txt")

In [ ]:
#I was not able to apply the map function for some reason (to get my tokenized dataset through the tokenizer function)
#And stackoverflow said that that function is for series as opposed to dataframes
# I tried converting pandas dataframe to series so that I could use the map function on it but it did not work

# print(datasetCBT_CN_test[:1])
# print(datasetCBT_CN_train.stack())
my_series1 = datasetCBT_CN_train['text'].squeeze()
my_series2 = datasetCBT_CN_train['labels'].squeeze()
my_series3 = my_series2.append(my_series1)
print(my_series3)
# datasetCBT_CN_train1 = Dataset.from_pandas(datasetCBT_CN_train)

In [ ]:
print("building tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# def tokenize_function(examples):
# 	# print(type(examples["context"]), type(examples["query"]), type(examples["candidate"]))
# 	token = tokenizer(examples["context"], examples["query"], examples["candidate"], padding="max_length", truncation=True)
# 	return token
def tokenize_function(examples):
	print(examples["text"])
	token = tokenizer(examples["text"], padding="max_length", truncation=True)
	return token

building tokenizer...


In [ ]:
tokenized_datasetCBT_CN_train = datasetCBT_CN_train.map(tokenize_function, batched=True)
print(tokenized_datasetCBT_CN_train)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length")


  0%|          | 0/121 [00:00<?, ?ba/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



ValueError: ignored

In [ ]:
#I also tried to use simpletransformers but realized quickly that I do not understand enough about 
#this architecture and I did not have the time to familiarize myself with it

# !pip install simpletransformers
# import logging
# from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

# # tokenized_datasetCBT_train = datasetCBT.map(tokenize_function, batched=True)
# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.WARNING)

# # Configure the model
# model_args = QuestionAnsweringArgs()
# model_args.train_batch_size = 8
# model_args.evaluate_during_training = True

# # Create the QuestionAnsweringModel
# model = QuestionAnsweringModel(
#     "distilbert", "distilbert-base-uncased",
#     args=model_args
# )

# # Train the model
# model.train_model(datasetCBT_CN_train, eval_data=datasetCBT_CN_validate)

# # Evaluate the model. (Being lazy and evaluating on the train data itself)
# result, text = model.eval_model(datasetCBT_CN_test)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this mode

KeyError: ignored